In [1]:
import os
import sys
import json
import time 
import enum
import inspect
import requests
import typing
import queue
import hashlib
import base64
import asyncio
import sqlite3
import subprocess as sp
from contextlib import closing

from dataclasses import dataclass, field
from typing import Callable, List, Union, Set, Any

import pymysql
import psycopg2
import psycopg2.extras

local_import_path = '/home/zu/x1ws/l6sk_python'

if local_import_path not in sys.path:
    sys.path.append(local_import_path)


In [2]:
from l6sk import knobman as km

In [3]:
_DEV_SECRET = sp.check_output("cat ~/.ssh/id_rsa.pub | sha512sum - | cut -c1-24", shell=True).decode().strip()

# dsn = "... dbname=postgres options='-c synchronous_commit=off'"
_PG_DSN_SYNC = {
    "database": "postgres",
    "host": "127.0.0.1",
    "port": "9832",
    "user": "postgres",
    "password": _DEV_SECRET,
    
    # any other options should be passed along to underlying lib, but only those psycopg2 understands
    # and accepts. "nonz": "hello world" wouldnt be accepted or passed along
    "application_name": "fts_ntbk_sync",
}

_PG_DSN_NO_SYNC = _PG_DSN_SYNC.copy() # shallow is enof.
_PG_DSN_NO_SYNC["options"] = "-c synchronous_commit=off"
_PG_DSN_NO_SYNC["application_name"] = "fts_ntbk_unsync"

In [4]:
unsync_dbconn = psycopg2.connect(**_PG_DSN_NO_SYNC)
unsync_dbconn.autocommit = True

In [5]:
# common q exec.
def exec_q(q, pr_res=True, col_names=True):
    with closing(unsync_dbconn.cursor()) as tmp_cur:
        tmp_cur.execute(q)
        if pr_res:
            if col_names:
                print("|".join([desc[0] for desc in tmp_cur.description]))
            print('-'*40)
            for row in tmp_cur.fetchall():
                print(row)
            # sep
            print("")

In [6]:
exec_q("SHOW synchronous_commit;")

synchronous_commit
----------------------------------------
('off',)



# test some competing table designs.